In [193]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from causalml.dataset import synthetic_data
from sklearn.model_selection import KFold

In [194]:
from importlib import reload
import causal_inference.propensity as causal

reload(causal)

<module 'causal_inference.propensity' from '/home/adam/Projects/causal_inference/causal_inference/propensity.py'>

In [195]:
def calculate_ate(ite):
    return ite.mean().round(2), ite.std().round(2)

def calculate_propensity(propensity):
    return propensity.mean().round(2), propensity.std().round(2)

In [196]:

y, X, treatment, true_ite, expected_outcome, true_propensity = synthetic_data(mode=1,
                                                                              n=1000,
                                                                              p=10,
                                                                              sigma=2)

# As the mean propensity doesn't say lot it would be nice to plot the true propensity to see the overlap between
# control and treated.
# It should be done like: http://ethen8181.github.io/machine-learning/ab_tests/causal_inference/matching.html

print("The true ATE of the generated data is",
      calculate_ate(true_ite)[0],
      "with standard deviation equal to",
      calculate_ate(true_ite)[1],
      ".")

print("The average propensity score value is equal to",
      calculate_propensity(true_propensity)[0],
      "with standard deviation equal to",
      calculate_propensity(true_propensity)[1],
      ".")

The true ATE of the generated data is 0.5 with standard deviation equal to 0.19 .
The average propensity score value is equal to 0.52 with standard deviation equal to 0.3 .


In [197]:
X_new = np.hstack((X, treatment.reshape(len(treatment), 1)))

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_new, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [198]:
treatment.astype(bool).dtype

dtype('bool')

In [199]:
from sklearn.linear_model import LinearRegression


model = LinearRegression()



In [200]:
my_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('estimate_propensity', causal.PropensityEstimator()),
    ('model', model)
])

In [201]:
from sklearn.metrics import mean_absolute_error

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

The propensity score was estimated using a logistic regression model with accuracy 0.73625
The balance of the covariate distribution is ?
(800, 11)
(800,)
The propensity score was estimated using a logistic regression model with accuracy 0.735
The balance of the covariate distribution is ?
(200, 11)
(200,)
MAE: 1.7725070910210758


In [202]:
# define the model cross-validation configuration

cv = KFold(n_splits=10, shuffle=True, random_state=1)

# evaluate the pipeline using cross validation and calculate MAE
scores = cross_val_score(my_pipeline, X_new, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# convert MAE scores to positive values
scores = absolute(scores)
# summarize the model performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

NameError: name 'cross_val_score' is not defined

NameError: name 'X_new' is not defined